# [LangChain](https://langchain.com/)

- opensource development framework for LLM applications
- Python and Javascript(TypeScript) packages

## Components
- Models
  * LLM: 20+ integrations
  * Chat Models
  * Text Embedding Models: 10+ integrations
- Prompts
  * Prompt templates
  * Output parsers: 5+ implementations
  * Example Selectors: 5+ implementations
- Indexes
  * Document loaders: 50+ implementations
  * Text Splitters: 10+
  * Vector stores:10+
  * Retrievers: 5+
- Chains
  * Prompt + LLM + Output parsing
  * Can be used as building blocks for longer chains
  * More application-specific chains: 20+ types
- Agents
  * Agent Types: 5+ types
  * Agent Tookits: 10+ 

# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [15]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [13]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [14]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [16]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [17]:
style = """Chinese \
in a calm and respectful tone
"""

In [18]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is Chinese in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [19]:
response = get_completion(prompt)
response

'哎呀，我的搅拌机盖飞了，把我的厨房墙壁都弄得满是果汁！更糟糕的是，保修不包括清洁厨房的费用。望你能立即帮我，伙计！'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
# current version: langchain-0.0.191
#!pip install --upgrade langchain

### Model

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
chat = ChatOpenAI(temperature=0.0) # 较高的 temperature 对于需要多样性或创造力的任务可能很有用，或者如果您想为最终用户生成更多而不同结果以供选择。
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

### Prompt template

In [22]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [23]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [24]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [25]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [26]:
customer_style = """Chinese \
in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [28]:
customer_message = prompt_template.format_messages(
    style=customer_style,
    text=customer_email)

In [29]:
print(type(customer_message))
print(type(customer_message[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [30]:
print(customer_message[0])

content="Translate the text that is delimited by triple backticks into a style that is Chinese in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


#### Call the LLM to translate to the style of the customer message

In [32]:
customer_response = chat(customer_message)

In [33]:
print(customer_response.content)

嗯，我很生气，我的搅拌机盖飞了出去，把我的厨房墙壁都弄得满是果汁！更糟糕的是，保修不包括清洁厨房的费用。伙计，我现在需要你的帮助！


In [34]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [35]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [36]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [37]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [38]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [40]:
customer_review = """\
送妈妈的母亲节礼物。妈妈之前用的手机还是差不多4、5年前的老款。\
不过这里插一嘴，iPhone还是挺耐用的。4、5年前的机型现在用也基本不卡，\
除了电池以外别的性能都还好。这次这个紫色的14真是太貌美了！\
颜色很有内涵，机身很有质感。希望能一如既往的好用！
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [41]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [43]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": -1,
    "price_value": ["这次这个紫色的14真是太貌美了！"]
}


In [44]:
type(response.content)

str

### Parse the LLM output string into a Python dictionary

In [45]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [46]:
gift_schema = ResponseSchema(name="gift", 
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schema = [gift_schema, delivery_days_schema, price_value_schema]

In [47]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [48]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [50]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review,
                                  format_instructions = format_instructions)

In [51]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: 送妈妈的母亲节礼物。妈妈之前用的手机还是差不多4、5年前的老款。不过这里插一嘴，iPhone还是挺耐用的。4、5年前的机型现在用也基本不卡，除了电池以外别的性能都还好。这次这个紫色的14真是太貌美了！颜色很有内涵，机身很有质感。希望能一如既往的好用！


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                  

In [52]:
response = chat(messages)
print(response.content)

```json
{
	"gift": false,
	"delivery_days": -1,
	"price_value": ["这次这个紫色的14真是太貌美了！"]
}
```


In [53]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False, 'delivery_days': -1, 'price_value': ['这次这个紫色的14真是太貌美了！']}

In [54]:
type(output_dict)

dict

In [55]:
output_dict.get('delivery_days')

-1